In [ ]:
import pickle
import h5py
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
import torch
import torchvision
import time

from utils import *

In [ ]:
# Set seeds for random generators for reproducability of results

random_seed = 100
torch.manual_seed(random_seed)
np.random.seed(random_seed)

# Dataset Preparation

The cells below will load the data from downloaded CIFAR-10 dataset and extract 800 train images and 200 test images per each class. The last cell will show a sample of 5 images per each class.

In [ ]:
base_dir = '../data/cifar-10-batches-py/'
data_train, labels_train = load_cifar_data([base_dir + 'data_batch_1', base_dir + 'data_batch_2', base_dir + 'data_batch_3'])
data_test, labels_test = load_cifar_data([base_dir + 'test_batch'])

train_indices = get_classwise_indices(labels_train)
test_indices = get_classwise_indices(labels_test)

train_data, train_labels = get_data_from_indices(data_train, train_indices, 800, (3,32,32))
test_data, test_labels = get_data_from_indices(data_test, test_indices, 200, (3,32,32))

labels = ['aiplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

train_loader = create_data_loader(train_data, train_labels, 40, True) 
test_loader = create_data_loader(test_data, test_labels, 40, False)

show_classwise_images(train_data, train_labels, labels, 5)

In [ ]:
# Download train and test data to computer to know data sizes

np.save("../data/train_data/train_data",train_data)
np.save("../data/train_data/train_image", train_data[0])
np.save("../data/test_data/test_data", test_data)
np.save("../data/train_data/train_labels", train_labels)
np.save("../data/test_data/test_labels", test_labels)

# Model Definitions

This section is used for defining all the models that will be used for this assignment. This section also prints the number trainable parameters for each model

In [ ]:
resnet = torchvision.models.resnet18(num_classes=10)
resnet34 = torchvision.models.resnet34(num_classes=10)
densenet = torchvision.models.densenet121(num_classes=10)

resnet_total_params = sum(p.numel() for p in resnet.parameters() if p.requires_grad)
resnet34_total_params = sum(p.numel() for p in resnet34.parameters() if p.requires_grad)
densenet_total_params = sum(p.numel() for p in densenet.parameters() if p.requires_grad)

print("Total parameters in ResNet-18:", resnet_total_params)
print("Total parameters in ResNet-34:", resnet34_total_params)
print("Total parameters in Densenet:", densenet_total_params)

# Training 4 CNNs for comparison of results

The below modules train 4 CNN classifiers and produce results like train, test time and accuracy, size of models in MB etc.

In [ ]:
num_epochs = 20
learning_rate = 0.001

In [ ]:
# Training baseline ResNet model
resnet_train_acc, resnet_test_acc, resnet_train_losses = train_model(resnet, train_loader, test_loader, num_epochs=num_epochs, learning_rate=learning_rate, save_epochs=[5,8,10,15,20], model_name='resnet18')

In [ ]:
print("Calculating Final Testing Accuracies:")
accuracy, time_taken = get_accuracies(test_loader, resnet)
print("Time taken for testing on 2000 images:", str(time_taken))
print("Time taken for each image:", str(time_taken / 2000))

In [ ]:
# Training ResNet-34 model
num_epochs = 5
resnet34_train_acc, resnet34_test_acc, resnet34_train_losses = train_model(resnet34, train_loader, test_loader, num_epochs=num_epochs, learning_rate=learning_rate, save_epochs=[5], model_name='resnet34')

print("Calculating Final Testing Accuracies:")
accuracy, time_taken = get_accuracies(test_loader, resnet34)
print("Time taken for testing on 2000 images:", str(time_taken))
print("Time taken for each image:", str(time_taken / 2000))

In [ ]:
# Training DenseNet-121 model
num_epochs = 5
densenet_train_acc, densenet_test_acc, densenet_train_losses = train_model(densenet, train_loader, test_loader, num_epochs=num_epochs, learning_rate=learning_rate, save_epochs=[5], model_name='squeezenet')

print("Calculating Final Testing Accuracies:")
accuracy, time_taken = get_accuracies(test_loader, outlier_densenet)
print("Time taken for testing on 2000 images:", str(time_taken))
print("Time taken for each image:", str(time_taken / 2000))

In [ ]:
# resnet_train_acc = [33.275 , 41.1625, 45.2875, 52.3, 55.1125, 57.425, 63.9375, 65.8875, 75.525, 77.2375, 84.0125, 88.5125, 91.45, 88.9875, 91.7625, 94.7125, 95.4375, 97.2875, 95.3375, 97.0375]
# resnet_test_acc = [32.45, 39.8, 42.05, 46.9, 47.0, 49.05, 50.7, 49.5, 52.1, 51.85, 51.5, 53.5, 51.5, 49.5, 50.75, 51.45, 50.0, 52.35, 50.9, 52.2]
# resnet34_train_acc = [30.125, 16.825, 25.9625, 38.5375, 42.2]
# resnet34_test_acc = [27.4, 16.95, 26.5, 37.85, 41.05]
# densenet_train_acc = [21.675, 36.8375, 37.325, 42.025, 49.9875]
# densenet_test_acc = [21.2, 35.65, 36.05, 39.45, 47.2]


plot_values(np.arange(1,21), [resnet_train_acc, resnet_test_acc], "Epoch", "Accuracy", "ResNet-18 Accuracy (20 epochs)", ["Train", "Test"], "resnet-16-20")
plot_values(np.arange(1,6), [resnet_train_acc[:5], resnet_test_acc[:5]], "Epoch", "Accuracy", "ResNet-18 Accuracy (5 epochs)", ["Train", "Test"], "resnet-16-5")
plot_values(np.arange(1,9), [resnet_train_acc[:8], resnet_test_acc[:8]], "Epoch", "Accuracy", "ResNet-18 Accuracy (8 epochs)", ["Train", "Test"], "resnet-16-8")
plot_values(np.arange(1,11), [resnet_train_acc[:10], resnet_test_acc[:10]], "Epoch", "Accuracy", "ResNet-18 Accuracy (10 epochs)", ["Train", "Test"], "resnet-16-10")
plot_values(np.arange(1,16), [resnet_train_acc[:15], resnet_test_acc[:15]], "Epoch", "Accuracy", "ResNet-18 Accuracy (15 epochs)", ["Train", "Test"], "resnet-16-15")
plot_values(np.arange(1,6), [resnet34_train_acc, resnet34_test_acc], "Epoch", "Accuracy", "ResNet-34 Accuracy (5 epochs)", ["Train", "Test"], "resnet-34")
plot_values(np.arange(1,6), [densenet_train_acc, densenet_test_acc], "Epoch", "Accuracy", "DenseNet Accuracy (5 epochs)", ["Train", "Test"], "DenseNet")

# Outlier Analysis

This section extracts images from CIFAR-100 dataset and performs outlier analysis on these images

In [ ]:
meta = unpickle('../data/cifar-100-python/meta')

fine_label_names = [t.decode('utf8') for t in meta[b'fine_label_names']]
cifar10_label_names = ['aiplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

train = unpickle('../data/cifar-100-python/train')

filenames = [t.decode('utf8') for t in train[b'filenames']]
fine_labels = train[b'fine_labels']
data = train[b'data']

outliers = []
outlier_labels = []

for idx, label in enumerate(fine_labels):
    if fine_label_names[label] not in cifar10_label_names:
        outliers.append(data[idx])
        outlier_labels.append(label)
    if(len(outliers) == 10):
        break

outliers = [np.reshape(image, (3,32,32)) for image in outliers]
outliers_tensor = torch.stack([torch.Tensor(i) for i in outliers])
outlier_label_names = [fine_label_names[label] for label in outlier_labels]

imshow(torchvision.utils.make_grid(outliers_tensor, nrow=1), outlier_label_names, "../data/outlier")

In [ ]:
# ResNet-18, 5 epochs

outlier_resnet = torch.load("../data/models/resnet18_5")
outlier_analysis(outlier_resnet, outliers_tensor, outlier_label_names, cifar10_label_names)

In [ ]:
# ResNet-18, 8 epochs

outlier_resnet = torch.load("../data/models/resnet18_8")
outlier_analysis(outlier_resnet, outliers_tensor, outlier_label_names, cifar10_label_names)

In [ ]:
# ResNet-18, 10 epochs

outlier_resnet = torch.load("../data/models/resnet18_10")
outlier_analysis(outlier_resnet, outliers_tensor, outlier_label_names, cifar10_label_names)

In [ ]:
# ResNet-18, 15 epochs

outlier_resnet = torch.load("../data/models/resnet18_15")
outlier_analysis(outlier_resnet, outliers_tensor, outlier_label_names, cifar10_label_names)

In [ ]:
# ResNet-18, 20 epochs

outlier_resnet = torch.load("../data/models/resnet18_20")
outlier_analysis(outlier_resnet, outliers_tensor, outlier_label_names, cifar10_label_names)

In [ ]:
# ResNet-34, 5 epochs

outlier_resnet = torch.load("../data/models/resnet34_5")
outlier_analysis(outlier_resnet, outliers_tensor, outlier_label_names, cifar10_label_names)

In [ ]:
# DenseNet, 5 epochs

outlier_densenet = torch.load("../data/models/densenet_5")
outlier_analysis(outlier_densenet, outliers_tensor, outlier_label_names, cifar10_label_names)